In [1]:
import os
import utils
import subprocess
import random
from subprocess import  check_output, CalledProcessError, STDOUT 
import multiprocessing

# Target dataset processor

#### Add names of target datasets to process here

In [2]:
datasetnames = ["YourName"] #folders containing datasets videos
extensions = ('.mp4', '.mkv', '.avi') #acceptable video extensions
exclusions = [] #original videos to not process, by filename

In [20]:
#split all videos in each dataset folder into scenes, dump scenes into unified dest folder
for datasetname in datasetnames:
    videolist = os.listdir("data/videos/tgts/" + datasetname)
    for videoname in videolist:
        if not videoname.lower().endswith(extensions) or videoname in exclusions:
            continue
        command = ("scenedetect -i " 
                  + "data/videos/tgts/" + datasetname + "/" + videoname + " "
                  + "-o " + "data/videos/tgts_scenes/" + datasetname + " " 
                  + "detect-content split-video")
        print(command)
        os.system(command)

scenedetect -i data/videos/tgts/YourName/YourName.mp4 -o data/videos/tgts_scenes/YourName detect-content split-video


In [ ]:
#split all scenes into frames, dump frames into dest dataset folder separated by scene
#ignore the first three frames of each scene for anime as scenedetect 
#seems to hold them from the previous scene

for datasetname in datasetnames:
    scenelist = os.listdir("data/videos/tgts_scenes/" + datasetname)
    
    try: #make dest folder if doesn't exist
        os.mkdir("data/AnimeGAN_data/target_datasets/" + datasetname)
    except OSError:
        pass
    
    for scene in scenelist:
        if not scene.lower().endswith(extensions): #skip junk files
            continue
        print("Splitting " + scene)
        inpath = "data/videos/tgts_scenes/" + datasetname + "/" + scene
        outfolder = "data/AnimeGAN_data/target_datasets/" + datasetname + "/" + scene
        frameoutfolder = "data/img_datasets/target_img_datasets/" + datasetname
        utils.FrameCapture(inpath, outfolder, div = 24, startFrame = 3)
        utils.FrameCapture(inpath, frameoutfolder, prefix = scene.split('.')[0],
                           div = 1, clearoutfolder = False)

Splitting YourName-Scene-0855.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-0855.mp4 
Splitting YourName-Scene-0581.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-0581.mp4 
Splitting YourName-Scene-0605.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-0605.mp4 
Splitting YourName-Scene-1253.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-1253.mp4 
Splitting YourName-Scene-0436.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-0436.mp4 
Splitting YourName-Scene-1229.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-1229.mp4 
Splitting YourName-Scene-0499.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-0499.mp4 
Splitting YourName-Scene-1469.mp4


Splitting YourName-Scene-1014.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-1014.mp4 
Splitting YourName-Scene-0759.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-0759.mp4 
Splitting YourName-Scene-1121.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-1121.mp4 
Splitting YourName-Scene-0844.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-0844.mp4 
Splitting YourName-Scene-0206.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-0206.mp4 
Splitting YourName-Scene-1438.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-1438.mp4 
Splitting YourName-Scene-0041.mp4
Successfully created the directory data/AnimeGAN_data/target_datasets/YourName/YourName-Scene-0041.mp4 
Splitting YourName-Scene-1329.mp4


In [ ]:
#TODO: EDGE-PROMOTING and CROPPING for both image and scenes

# Source dataset processor

In [3]:
#https://stackoverflow.com/questions/3844430/how-to-get-the-duration-of-a-video-in-python
def getDuration(filename):
    command = [
        'ffprobe', 
        '-v', 
        'error', 
        '-show_entries', 
        'format=duration', 
        '-of', 
        'default=noprint_wrappers=1:nokey=1', 
        filename
      ]

    try:
        output = check_output( command, stderr=STDOUT ).decode()
    except CalledProcessError as e:
        output = e.output.decode()

    return output

In [4]:
def eraseWatermark(tup):
    videoname = tup
    if not videoname.lower().endswith(('.avi')):
        return
    vidpath = "data/videos/src/" + videoname
    outvidname = "data/videos/src/" + videoname.split('.')[0] + ".mp4"
    try:
        length = float(getDuration(vidpath))
    except:
        os.system("rm " + vidpath)
        return
    if length < 12.0:
        os.system("rm " + vidpath)
        return
    ccommand = "ffmpeg -i " + vidpath + " -ss 10 " + outvidname
    print(ccommand)
    os.system(ccommand)
    os.system("rm " + vidpath)
    
videolist = os.listdir("data/videos/src/")
pool = multiprocessing.Pool()
args = [videoname for videoname in videolist]
pool.imap_unordered(eraseWatermark, args)
pool.close()
pool.join()

ffmpeg -i data/videos/src/actioncliptrain00131.avi -ss 10 data/videos/src/actioncliptrain00131.mp4
ffmpeg -i data/videos/src/actioncliptest00585.avi -ss 10 data/videos/src/actioncliptest00585.mp4
ffmpeg -i data/videos/src/scenecliptest00257.avi -ss 10 data/videos/src/scenecliptest00257.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00490.avi -ss 10 data/videos/src/sceneclipautoautotrain00490.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00003.avi -ss 10 data/videos/src/actionclipautoautotrain00003.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00028.avi -ss 10 data/videos/src/sceneclipautoautotrain00028.mp4
ffmpeg -i data/videos/src/scenecliptest00450.avi -ss 10 data/videos/src/scenecliptest00450.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00377.avi -ss 10 data/videos/src/sceneclipautoautotrain00377.mp4
ffmpeg -i data/videos/src/scenecliptest00250.avi -ss 10 data/videos/src/scenecliptest00250.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00098.avi -ss 10 

ffmpeg -i data/videos/src/sceneclipautoautotrain00250.avi -ss 10 data/videos/src/sceneclipautoautotrain00250.mp4
ffmpeg -i data/videos/src/scenecliptest00463.avi -ss 10 data/videos/src/scenecliptest00463.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00301.avi -ss 10 data/videos/src/sceneclipautoautotrain00301.mp4
ffmpeg -i data/videos/src/scenecliptest00341.avi -ss 10 data/videos/src/scenecliptest00341.mp4
ffmpeg -i data/videos/src/scenecliptest00330.avi -ss 10 data/videos/src/scenecliptest00330.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00075.avi -ss 10 data/videos/src/sceneclipautoautotrain00075.mp4
ffmpeg -i data/videos/src/actioncliptest00564.avi -ss 10 data/videos/src/actioncliptest00564.mp4
ffmpeg -i data/videos/src/actioncliptrain00517.avi -ss 10 data/videos/src/actioncliptrain00517.mp4
ffmpeg -i data/videos/src/scenecliptest00105.avi -ss 10 data/videos/src/scenecliptest00105.mp4
ffmpeg -i data/videos/src/actioncliptest00604.avi -ss 10 data/videos/src/actionclipte

ffmpeg -i data/videos/src/sceneclipautoautotrain00003.avi -ss 10 data/videos/src/sceneclipautoautotrain00003.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00765.avi -ss 10 data/videos/src/actionclipautoautotrain00765.mp4
ffmpeg -i data/videos/src/actioncliptest00384.avi -ss 10 data/videos/src/actioncliptest00384.mp4
ffmpeg -i data/videos/src/actioncliptest00618.avi -ss 10 data/videos/src/actioncliptest00618.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00554.avi -ss 10 data/videos/src/actionclipautoautotrain00554.mp4
ffmpeg -i data/videos/src/actioncliptest00128.avi -ss 10 data/videos/src/actioncliptest00128.mp4
ffmpeg -i data/videos/src/actioncliptest00022.avi -ss 10 data/videos/src/actioncliptest00022.mp4
ffmpeg -i data/videos/src/actioncliptrain00321.avi -ss 10 data/videos/src/actioncliptrain00321.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00158.avi -ss 10 data/videos/src/actionclipautoautotrain00158.mp4
ffmpeg -i data/videos/src/scenecliptest00168.avi -ss 10

ffmpeg -i data/videos/src/sceneclipautoautotrain00199.avi -ss 10 data/videos/src/sceneclipautoautotrain00199.mp4
ffmpeg -i data/videos/src/scenecliptest00391.avi -ss 10 data/videos/src/scenecliptest00391.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00050.avi -ss 10 data/videos/src/sceneclipautoautotrain00050.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00530.avi -ss 10 data/videos/src/sceneclipautoautotrain00530.mp4
ffmpeg -i data/videos/src/scenecliptest00461.avi -ss 10 data/videos/src/scenecliptest00461.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00566.avi -ss 10 data/videos/src/actionclipautoautotrain00566.mp4
ffmpeg -i data/videos/src/actioncliptrain00124.avi -ss 10 data/videos/src/actioncliptrain00124.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00432.avi -ss 10 data/videos/src/sceneclipautoautotrain00432.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00368.avi -ss 10 data/videos/src/actionclipautoautotrain00368.mp4
ffmpeg -i data/videos/src/sce

In [ ]:
# videolist = os.listdir("data/videos/src/")
# for videoname in videolist:
#     if not videoname.lower().endswith(('.avi')):
#         continue
#     vidpath = "data/videos/src/" + videoname
#     outvidname = "data/videos/src/" + videoname.split('.')[0] + ".mp4"
#     try:
#         length = float(getDuration(vidpath))
#     except:
#         os.system("rm " + vidpath)
#         continue
#     if length < 12.0:
#         os.system("rm " + vidpath)
#         continue
#     ccommand = "ffmpeg -i " + vidpath + " -ss 10 " + outvidname
#     print(ccommand)
#     os.system(ccommand)
#     os.system("rm " + vidpath)

ffmpeg -i data/videos/src/scenecliptest00174.avi -ss 10 data/videos/src/scenecliptest00174.mp4
ffmpeg -i data/videos/src/scenecliptest00028.avi -ss 10 data/videos/src/scenecliptest00028.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00080.avi -ss 10 data/videos/src/sceneclipautoautotrain00080.mp4
ffmpeg -i data/videos/src/scenecliptest00039.avi -ss 10 data/videos/src/scenecliptest00039.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00110.avi -ss 10 data/videos/src/sceneclipautoautotrain00110.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00468.avi -ss 10 data/videos/src/sceneclipautoautotrain00468.mp4
ffmpeg -i data/videos/src/scenecliptest00511.avi -ss 10 data/videos/src/scenecliptest00511.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00651.avi -ss 10 data/videos/src/actionclipautoautotrain00651.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00336.avi -ss 10 data/videos/src/actionclipautoautotrain00336.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain005

ffmpeg -i data/videos/src/scenecliptest00236.avi -ss 10 data/videos/src/scenecliptest00236.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00091.avi -ss 10 data/videos/src/sceneclipautoautotrain00091.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00478.avi -ss 10 data/videos/src/sceneclipautoautotrain00478.mp4
ffmpeg -i data/videos/src/actioncliptrain00429.avi -ss 10 data/videos/src/actioncliptrain00429.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00487.avi -ss 10 data/videos/src/actionclipautoautotrain00487.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00307.avi -ss 10 data/videos/src/actionclipautoautotrain00307.mp4
ffmpeg -i data/videos/src/scenecliptest00120.avi -ss 10 data/videos/src/scenecliptest00120.mp4
ffmpeg -i data/videos/src/scenecliptest00374.avi -ss 10 data/videos/src/scenecliptest00374.mp4
ffmpeg -i data/videos/src/scenecliptest00041.avi -ss 10 data/videos/src/scenecliptest00041.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00017.avi -ss 10 

ffmpeg -i data/videos/src/actioncliptest00181.avi -ss 10 data/videos/src/actioncliptest00181.mp4
ffmpeg -i data/videos/src/actioncliptrain00239.avi -ss 10 data/videos/src/actioncliptrain00239.mp4
ffmpeg -i data/videos/src/scenecliptest00210.avi -ss 10 data/videos/src/scenecliptest00210.mp4
ffmpeg -i data/videos/src/scenecliptest00327.avi -ss 10 data/videos/src/scenecliptest00327.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00243.avi -ss 10 data/videos/src/sceneclipautoautotrain00243.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00082.avi -ss 10 data/videos/src/sceneclipautoautotrain00082.mp4
ffmpeg -i data/videos/src/scenecliptest00362.avi -ss 10 data/videos/src/scenecliptest00362.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00163.avi -ss 10 data/videos/src/actionclipautoautotrain00163.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00053.avi -ss 10 data/videos/src/sceneclipautoautotrain00053.mp4
ffmpeg -i data/videos/src/actioncliptest00424.avi -ss 10 data/vid

ffmpeg -i data/videos/src/scenecliptest00248.avi -ss 10 data/videos/src/scenecliptest00248.mp4
ffmpeg -i data/videos/src/actioncliptest00822.avi -ss 10 data/videos/src/actioncliptest00822.mp4
ffmpeg -i data/videos/src/scenecliptest00031.avi -ss 10 data/videos/src/scenecliptest00031.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00569.avi -ss 10 data/videos/src/actionclipautoautotrain00569.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00254.avi -ss 10 data/videos/src/sceneclipautoautotrain00254.mp4
ffmpeg -i data/videos/src/scenecliptest00073.avi -ss 10 data/videos/src/scenecliptest00073.mp4
ffmpeg -i data/videos/src/scenecliptest00247.avi -ss 10 data/videos/src/scenecliptest00247.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00566.avi -ss 10 data/videos/src/sceneclipautoautotrain00566.mp4
ffmpeg -i data/videos/src/actioncliptest00347.avi -ss 10 data/videos/src/actioncliptest00347.mp4
ffmpeg -i data/videos/src/scenecliptest00133.avi -ss 10 data/videos/src/scenecliptest

ffmpeg -i data/videos/src/sceneclipautoautotrain00397.avi -ss 10 data/videos/src/sceneclipautoautotrain00397.mp4
ffmpeg -i data/videos/src/actioncliptest00816.avi -ss 10 data/videos/src/actioncliptest00816.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00211.avi -ss 10 data/videos/src/actionclipautoautotrain00211.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00260.avi -ss 10 data/videos/src/actionclipautoautotrain00260.mp4
ffmpeg -i data/videos/src/scenecliptest00501.avi -ss 10 data/videos/src/scenecliptest00501.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00470.avi -ss 10 data/videos/src/sceneclipautoautotrain00470.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00042.avi -ss 10 data/videos/src/actionclipautoautotrain00042.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00236.avi -ss 10 data/videos/src/sceneclipautoautotrain00236.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00196.avi -ss 10 data/videos/src/actionclipautoautotrain00196.mp4
ffmpeg -i

...

ffmpeg -i data/videos/src/actioncliptest00753.avi -ss 10 data/videos/src/actioncliptest00753.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00770.avi -ss 10 data/videos/src/actionclipautoautotrain00770.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00782.avi -ss 10 data/videos/src/actionclipautoautotrain00782.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00140.avi -ss 10 data/videos/src/sceneclipautoautotrain00140.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00485.avi -ss 10 data/videos/src/actionclipautoautotrain00485.mp4
ffmpeg -i data/videos/src/actioncliptrain00814.avi -ss 10 data/videos/src/actioncliptrain00814.mp4
ffmpeg -i data/videos/src/scenecliptest00406.avi -ss 10 data/videos/src/scenecliptest00406.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00189.avi -ss 10 data/videos/src/sceneclipautoautotrain00189.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00622.avi -ss 10 data/videos/src/actionclipautoautotrain00622.mp4
ffmpeg -i data/videos/s

ffmpeg -i data/videos/src/sceneclipautoautotrain00379.avi -ss 10 data/videos/src/sceneclipautoautotrain00379.mp4
ffmpeg -i data/videos/src/actioncliptrain00665.avi -ss 10 data/videos/src/actioncliptrain00665.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00501.avi -ss 10 data/videos/src/sceneclipautoautotrain00501.mp4
ffmpeg -i data/videos/src/actioncliptest00020.avi -ss 10 data/videos/src/actioncliptest00020.mp4
ffmpeg -i data/videos/src/scenecliptest00196.avi -ss 10 data/videos/src/scenecliptest00196.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00212.avi -ss 10 data/videos/src/sceneclipautoautotrain00212.mp4
ffmpeg -i data/videos/src/actioncliptrain00432.avi -ss 10 data/videos/src/actioncliptrain00432.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00106.avi -ss 10 data/videos/src/sceneclipautoautotrain00106.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00283.avi -ss 10 data/videos/src/actionclipautoautotrain00283.mp4
ffmpeg -i data/videos/src/actionclipautoaut

ffmpeg -i data/videos/src/sceneclipautoautotrain00136.avi -ss 10 data/videos/src/sceneclipautoautotrain00136.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00429.avi -ss 10 data/videos/src/sceneclipautoautotrain00429.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00038.avi -ss 10 data/videos/src/actionclipautoautotrain00038.mp4
ffmpeg -i data/videos/src/actioncliptest00539.avi -ss 10 data/videos/src/actioncliptest00539.mp4
ffmpeg -i data/videos/src/scenecliptest00165.avi -ss 10 data/videos/src/scenecliptest00165.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00188.avi -ss 10 data/videos/src/sceneclipautoautotrain00188.mp4
ffmpeg -i data/videos/src/actioncliptest00554.avi -ss 10 data/videos/src/actioncliptest00554.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00369.avi -ss 10 data/videos/src/actionclipautoautotrain00369.mp4
ffmpeg -i data/videos/src/actioncliptest00401.avi -ss 10 data/videos/src/actioncliptest00401.mp4
ffmpeg -i data/videos/src/actionclipautoautot

ffmpeg -i data/videos/src/actionclipautoautotrain00386.avi -ss 10 data/videos/src/actionclipautoautotrain00386.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00292.avi -ss 10 data/videos/src/sceneclipautoautotrain00292.mp4
ffmpeg -i data/videos/src/scenecliptest00003.avi -ss 10 data/videos/src/scenecliptest00003.mp4
ffmpeg -i data/videos/src/actioncliptest00549.avi -ss 10 data/videos/src/actioncliptest00549.mp4
ffmpeg -i data/videos/src/actioncliptest00171.avi -ss 10 data/videos/src/actioncliptest00171.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00049.avi -ss 10 data/videos/src/actionclipautoautotrain00049.mp4
ffmpeg -i data/videos/src/scenecliptest00551.avi -ss 10 data/videos/src/scenecliptest00551.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00049.avi -ss 10 data/videos/src/sceneclipautoautotrain00049.mp4
ffmpeg -i data/videos/src/actioncliptrain00194.avi -ss 10 data/videos/src/actioncliptrain00194.mp4
ffmpeg -i data/videos/src/actioncliptrain00428.avi -ss 10 dat

ffmpeg -i data/videos/src/actioncliptrain00732.avi -ss 10 data/videos/src/actioncliptrain00732.mp4
ffmpeg -i data/videos/src/actioncliptrain00753.avi -ss 10 data/videos/src/actioncliptrain00753.mp4
ffmpeg -i data/videos/src/scenecliptest00502.avi -ss 10 data/videos/src/scenecliptest00502.mp4
ffmpeg -i data/videos/src/actioncliptest00162.avi -ss 10 data/videos/src/actioncliptest00162.mp4
ffmpeg -i data/videos/src/actioncliptest00163.avi -ss 10 data/videos/src/actioncliptest00163.mp4
ffmpeg -i data/videos/src/sceneclipautoautotrain00392.avi -ss 10 data/videos/src/sceneclipautoautotrain00392.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00402.avi -ss 10 data/videos/src/actionclipautoautotrain00402.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00125.avi -ss 10 data/videos/src/actionclipautoautotrain00125.mp4
ffmpeg -i data/videos/src/actioncliptrain00795.avi -ss 10 data/videos/src/actioncliptrain00795.mp4
ffmpeg -i data/videos/src/actionclipautoautotrain00099.avi -ss 10 data/

In [ ]:
videolist = os.listdir("data/videos/src/")
for videoname in videolist:
    if not videoname.lower().endswith(('.mp4')):
        continue
    command = ("scenedetect -i " 
              + "data/videos/src/" + videoname + " "
              + "-o " + "data/videos/src_scenes" + " " 
              + "detect-content split-video")
    print(command)
    os.system(command)

In [ ]:
scenelist = os.listdir("data/videos/src_scenes/")
    
for scene in scenelist:
    if not scene.lower().endswith(extensions): #skip junk files
        continue
    print("Splitting " + scene)
    inpath = "data/videos/src_scenes/" + scene
    outfolder = "data/AnimeGAN_data/src_data/" + scene
    frameoutfolder = "data/img_datasets/src_img/"
    utils.FrameCapture(inpath, outfolder, div = 24, startframe = 240)
    utils.FrameCapture(inpath, frameoutfolder, prefix = scene.split('.')[0],
                       div = 1, startframe = 10, clearoutfolder = False)

In [ ]:
#TODO: CROPPING for both image and scenes